In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pystan

%matplotlib inline
custom_style = {'axes.labelcolor': 'white',
                'xtick.color': 'white',
                'ytick.color': 'white'}
sns.set_style("darkgrid", rc=custom_style)
sns.set_context("notebook")
plt.style.use('dark_background')
plt.rcParams["font.size"] = 18
np.random.seed(123)

In [16]:
df = pd.read_csv('./data/ch6_2_2.csv', header=None)
df = df.T
df.columns = ['x1', 'x2']

In [19]:
df.shape

(20, 2)

In [133]:
stan_model = """
    data { 
        int<lower=0>  N;    
        vector[2] X[N];       
    }
    
    parameters {
        vector[2]           mu;
        vector<lower=0>[2]  sigma;
        real<lower=-1,upper=1>    rho;
    }
    
    transformed parameters {
        vector<lower=0>[2] sigmasq;
        matrix[2,2]      Sigma;

        sigmasq[1] = pow(sigma[1],2);
        sigmasq[2] = pow(sigma[2],2);
        Sigma[1,1] = sigmasq[1];
        Sigma[2,2] = sigmasq[2];
        Sigma[1,2] = sigma[1]*sigma[2]*rho;
        Sigma[2,1] = sigma[1]*sigma[2]*rho;
    }
    
    model {
        for(i in 1:N){
            X[i] ~ multi_normal(mu,Sigma);
        }
    }
    
    generated quantities{
        real delta;
        real delta_over;
        real delta_over2;
        real rho_over;
        real rho_over05;

        delta = mu[2] - mu[1];
        delta_over = step(delta);
        delta_over2 = delta>2 ? 1 : 0;
        rho_over = step(rho);
        rho_over05 = rho>0.5 ? 1 : 0;
    }
"""

In [111]:
sm = pystan.StanModel(model_code = stan_model2)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1d718054f5397f38db695f83708f77f9 NOW.


In [130]:
stan_data = {"N": df.shape[0], "X": np.array(df[['x1', 'x2']])}

In [131]:
fit = sm.sampling(data=stan_data, iter=11000, warmup=1000, chains=3, seed=1234)

In [132]:
fit

Inference for Stan model: anon_model_1d718054f5397f38db695f83708f77f9.
3 chains, each with iter=11000; warmup=1000; thin=1; 
post-warmup draws per chain=10000, total post-warmup draws=30000.

              mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu[0]        10.16  7.3e-3   0.89   8.42   9.58  10.15  10.72  11.95  15096    1.0
mu[1]        11.95  6.8e-3   0.85  10.26   11.4  11.95   12.5  13.64  15673    1.0
sigma[0]      3.87  5.7e-3   0.69    2.8   3.38   3.77   4.24   5.47  14836    1.0
sigma[1]      3.72  5.5e-3   0.66    2.7   3.26   3.64   4.09   5.28  14145    1.0
rho            0.7  9.8e-4   0.12    0.4   0.63   0.72   0.79   0.88  16052    1.0
sigmasq[0]   15.44    0.05   5.82   7.85  11.44  14.22  17.99  29.96  14027    1.0
sigmasq[1]    14.3    0.05   5.33    7.3  10.62  13.22  16.71  27.87  13315    1.0
Sigma[0,0]   15.44    0.05   5.82   7.85  11.44  14.22  17.99  29.96  14027    1.0
Sigma[1,0]   10.48    0.05    4.7   4.14   7.25   9.61   12.6